# Rates calculation

This notebooks demonstrates basic rates calculation method. We say "basic"
because we have also implemented a PyTorch module for rates calculation that can work on GPU. 

In [1]:
import sys
import numpy as np
import pandas as pd
from datetime import timedelta

sys.path.append('..')

from deepfield import Field

We need a model with computed states. Here we assume that the provided `norne_simplifed` model has been simulated with some external software and contains the RESULTS folder with states given in binary `.X` files or `UNRST` file. Let's load the model:

In [2]:
model = Field('../open_data/norne_simplified/norne_simplified.data').load()

INFO:Field:Using default config.
INFO:Field:Start reading X files.
INFO:Field:Start reading ../open_data/norne_simplified/RESULTS/norne_simplified.X0000
INFO:Field:Decoding PRESSURE.
INFO:Field:Decoding RS.
INFO:Field:Decoding SGAS.
INFO:Field:Decoding SOIL.
INFO:Field:Decoding SWAT.
INFO:Field:Finish reading ../open_data/norne_simplified/RESULTS/norne_simplified.X0000
INFO:Field:Start reading ../open_data/norne_simplified/RESULTS/norne_simplified.X0001
...
INFO:Field:===== Field summary =====
INFO:Field:GRID attributes: MAPAXES, ZCORN, COORD, ACTNUM, DIMENS
INFO:Field:ROCK attributes: PORO, PERMX, PERMY, PERMZ
INFO:Field:STATES attributes: PRESSURE, RS, SGAS, SOIL, SWAT
INFO:Field:TABLES attributes: PVTO, ROCK, PVTW, DENSITY, SWOF, SGOF, PVDG
INFO:Field:WELLS attributes: COMPDAT, WCONPROD, WELLTRACK, WELSPECS, RESULTS
INFO:Field:AQUIFERS attributes: 
INFO:Field:=========================
INFO:Field:Grid pillars (`COORD`) are mapped to new axis with respect to `MAPAXES`.


Required well's preprocessing:

In [3]:
(model.wells
 .drop_incomplete()
 .get_wellblocks(model.grid)
 .drop_outside()
 .apply_perforations()
 .calculate_cf(rock=model.rock, grid=model.grid)
 .compute_events(grid=model.grid))

Get rates (by default CPU multiprocessing is used):

In [4]:
model.calculate_rates()

Processed 36 out of 36 wells


Now we can check daily rates data:

In [5]:
model.wells.total_rates.iloc[15:20]

DATE     WOPR     WWPR         WGPR        WFGPR
15 1999-04-01        0        0            0            0
16 1999-05-17        0        0            0            0
17 1999-05-20  6571.39  21180.1  8.86762e+06  8.16128e+06
18 1999-06-22  5179.07    18197  6.09693e+06  5.54105e+06
19 1999-07-01  5022.09  17821.3  5.81483e+06    5.276e+06

and cumulative rates:

In [6]:
model.wells.cum_rates.iloc[15:20]

DATE     WOPR     WWPR         WGPR        WFGPR
15 1999-04-01        0        0            0            0
16 1999-05-17        0        0            0            0
17 1999-05-20  6571.39  21180.1  8.86762e+06  8.16128e+06
18 1999-06-22  11750.5  39377.1  1.49646e+07  1.37023e+07
19 1999-07-01  16772.5  57198.4  2.07794e+07  1.89783e+07

Note: to get detailed rates for each block see the `BLOCKS_DYNAMICS` attribute.

Rates can be quickly visualized using `show_rates` method:
```python
model.wells.show_rates()
```
It activates an interactive widget where one can specify wellname, phase, time interval and cumulative mode as shown below:

<p></p>

Note: to get detailed plots for each block use 
```python
model.wells.show_blocks_dynamics()
```

Done!